[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/on_drive/fast_sdwebui_on_drive.ipynb)

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/Van-wise/sd-colab


**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244


In [ ]:
#@title 👇 0.首次运行！ （只需 1 次）
#此单元格每个账号只需要运行一次，或用于更新。作用是部署SD，和安装常用插件。
import os
import sys

%cd /content
from google.colab import drive
drive.mount('/content/drive')

import binascii
sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
%env PYTHONDONTWRITEBYTECODE=1

#rsync sdwebui
!apt -y install -qq aria2
!apt --fix-broken install
#!apt -y install -qq aria2 git libcairo2-dev pkg-config python3-dev
#!git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw && git -C {webui_dir} reset --hard && git -C {webui_dir} pull
!git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw
!rsync -avqu {webui_dir} /content/drive/MyDrive/

#embeddings&positive&scripts
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {gwebui_dir}/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/$sdw-scripts/main/run_n_times.py -O {gwebui_dir}/scripts/run_n_times.py

!rm -rf {webui_dir}/embeddings/negative && rm -rf {webui_dir}/models/Lora/positive
!git clone https://huggingface.co/embed/negative {webui_dir}/embeddings/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' {webui_dir}/embeddings/negative {gwebui_dir}/embeddings
!git clone https://huggingface.co/embed/lora {webui_dir}/models/Lora/positive
!rsync -avqu --exclude '.git' --exclude '.gitattributes' {webui_dir}/models/Lora/positive {gwebui_dir}/models/Lora

!git clone https://github.com/AUTOMATIC1111/{saa}.git
!mv /content/{saa} {sd_dir}/repositories/ 

#插件&config
if not os.path.exists('/content/extensions.py'):
  !wget https://raw.githubusercontent.com/Van-wise/sd-colab/main/on_drive/extensions.py
!python /content/extensions.py

#自定义插件 ***=插件名称 xxx=插件项目地址
#!rm -rf {gwebui_dir}/extensions/***
#!git clone xxx {gwebui_dir}/extensions/***
#!git -C {gwebui_dir}/extensions/*** reset --hard && git -C {gwebui_dir}/extensions/*** pull

#CKPT Model Download
#gmodel_dir = f"{gwebui_dir}/models/Stable-diffusion"
#!wget https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -O {model_dir}/chilloutmix_NiPrunedFp32Fix.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/dawn6666/real-max-v3.4/resolve/main/real-max-v3.4.safetensors -d {gmodel_dir} -o real-max-v3.4.safetensors

#CLIP
if not os.path.exists(f"{gwebui_dir}/models/CLIP"):
  os.mkdir(f"{gwebui_dir}/models/CLIP")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt -d {gwebui_dir}/models/CLIP -o ViT-L-14.pt

%cd {gwebui_dir}
#!python launch.py --skip-torch-cuda-test

print("SDwebui on google drive.")

In [ ]:
#@title 👇 1.启动SDwebui！
#此单元格，每次连接colab都需要运行一次；作用是，安装运行环境并启动SDwebui。
import os
import sys
import glob
from IPython.utils import capture
from IPython.display import clear_output

import binascii
sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
saa = sdw + "-assets"

 
#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

!apt -y install -qq aria2
![ -d "{webui_dir}" ] || git clone -q https://github.com/AUTOMATIC1111/$sdw {webui_dir}
!pip install lmdb
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0 torchtext==0.16.0+cpu torchdata==0.7.0 --index-url https://download.pytorch.org/whl/cu118
# Requirements stable-diffusion-stability-ai
with capture.capture_output() as cap:
  %cd /content
  #!rm -rf {gwebui_dir}/repositories/stable-diffusion-stability-ai
  !mkdir -p {gwebui_dir}/repositories/stable-diffusion-stability-ai
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content/ --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  if not os.path.exists(f'{gwebui_dir}/libtcmalloc/libtcmalloc_minimal.so.4'):
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O {gwebui_dir}/libtcmalloc/libtcmalloc_minimal.so.4
    %env LD_PRELOAD={gwebui_dir}/libtcmalloc/libtcmalloc_minimal.so.4
  else:
    %env LD_PRELOAD={gwebui_dir}/libtcmalloc/libtcmalloc_minimal.so.4

  !rsync -avqu /content/sd/stablediffusion/* {gwebui_dir}/repositories/stable-diffusion-stability-ai && rm -rf /content/sd
  !git clone https://github.com/AUTOMATIC1111/{saa}.git
  !mv /content/{saa} {sd_dir}/repositories/ 
 
!mkdir -p {webui_dir}/cache
os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore'
!sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py
!apt --fix-broken install
%env PYTHONDONTWRITEBYTECODE=1
%env TF_CPP_MIN_LOG_LEVEL=1

#链接 controlnet
#@markdown 选择ControlNet：
#@markdown 
#@markdown 0：不需要使用ControlNet
#@markdown 
#@markdown 1：下载ControlNet 模型到 Colab
#@markdown 
#@markdown 2：下载ControlNet 模型到 Google drive    (会占用云盘,不推荐！)
%cd /content
controlnet = 0 #@param {type:"slider", min:0, max:2, step:1}
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"
!mkdir -p {cn_dir} && mkdir -p {gcn_dir} 
!test -d {cn_dir} && test -d {gcn_dir} && ln -sf {gcn_dir} {cn_dir}
!pip install -qq pycloudflared ngrok
!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv

if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
elif controlnet == 2:
    # 下载ControlNet 模型到 Google drive
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_drive
    cndown_drive()
else:
    print("不使用 ControlNet、或已使用快捷方式。")


#插件下载 extensions
#ex_dir = {webui_dir}/extensions
#!git clone --depth 1 https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {ex_dir }/$sdw-localization-zh_Hans #中文
#!git clone --depth 1 https://github.com/zanllp/$w-infinite-image-browsing {ex_dir }/$w-infinite-image-browsing #图片浏览

#链接 models
!test -d {webui_dir}/models/Stable-diffusion && test -d {gwebui_dir}/models/Stable-diffusion && ln -sf {gwebui_dir}/models/Stable-diffusion {webui_dir}/models/Stable-diffusion
model_dir = os.path.join(webui_dir, 'models', 'Stable-diffusion')
gmodel_dir = os.path.join(gwebui_dir, 'models', 'Stable-diffusion')
if not any(fname.endswith(('.safetensors', '.ckpt')) for fname in glob.glob(os.path.join(model_dir, '*'))) and not any(fname.endswith(('.safetensors', '.ckpt')) for fname in glob.glob(os.path.join(gmodel_dir, '*'))):
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors
else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors
#链接 Lora
!test -d {webui_dir}/models/Lora && test -d {gwebui_dir}/models/Lora && ln -sf {gwebui_dir}/models/Lora {webui_dir}/models/Lora
Lora_dir = os.path.join(gwebui_dir, 'models', 'Lora')
gLora_dir = os.path.join(gwebui_dir, 'models', 'Lora')
if not any(fname.endswith(('.safetensors', '.ckpt')) for fname in glob.glob(os.path.join(Lora_dir, '*'))) and not any(fname.endswith(('.safetensors', '.ckpt')) for fname in glob.glob(os.path.join(gLora_dir, '*'))):
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/62833 -d {webui_dir}/models/Lora -o Detail_Tweaker_LoRA.safetensors ##Detail Tweaker LoRA/

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {gwebui_dir}/modules
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {gwebui_dir}
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {gwebui_dir}/webui.py
    !sed -i 's@print(\"No module.*@@' /content/sd/stablediffusion/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"{gwebui_dir}/repositories/stable-diffusion-stability-ai\"]@' {gwebui_dir}/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' {gwebui_dir}/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' {gwebui_dir}/modules/paths.py

    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@"  {gwebui_dir}/modules/sd_models.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@"  {gwebui_dir}/modules/extras.py
wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark --skip-version-check"

#@markdown ####❎是否使用[ngrok](https://dashboard.ngrok.com/get-started/your-authtoken)网络加速
#@markdown ######填写ngrok_auth 的 token：
ngrok_auth=""  #@param {type:"string"}
if ngrok_auth:
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"

!python {gwebui_dir}/webui.py $wise --ckpt-dir {model_dir} --controlnet-dir {cn_dir}


In [ ]:
#@title 👇 1.1再次启动SDwebui！
#此单元格的用法是，在运行过1. 的前提下（安装好环境），启动SDweibui。
import os
import sys

import binascii
sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {gwebui_dir}/modules
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {gwebui_dir}
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {gwebui_dir}/webui.py
    !sed -i 's@print(\"No module.*@@' /content/sd/stablediffusion/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"/content/sd/stablediffusion/\"]@' {gwebui_dir}/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' {gwebui_dir}/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' {gwebui_dir}/modules/paths.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark --skip-version-check"
!python {gwebui_dir}/webui.py $wise --ckpt-dir {model_dir} --controlnet-dir {cn_dir}

In [ ]:
#@title 👇 附加模型快捷方式！
#附加模型的快捷方式，也就是使用别人分享的模型。附加过就不需要再次运行了！
import os
import sys
import binascii


sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
!rm -rf {gwebui_dir}/$sdw
# get save_dir_id, save_dir = save_dir_id
def get_save_dir_id (save_dir):

    save_dir_id = None
    try:
        path_parts = save_dir.split('/')
        parent_id = None
        for folder_name in path_parts[4:]:
            query = f"mimeType='application/vnd.google-apps.folder' and trashed = false and name='{folder_name}'"
            if parent_id:
                query += f" and '{parent_id}' in parents"
            results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
            folder = results.get('files', [])[0]
            save_dir_id = folder.get('id')
            parent_id = save_dir_id
    except HttpError as error:
        print(f'An error occurred: {error}')
    print(save_dir_id)
    return save_dir_id

# /models/Stable-diffusion/sdmodels
def add_shortcut_directory_to_drive (shortcut_name, share_id, save_dir_name, save_dir):

    query = f"name='{save_dir_name}' and mimeType='application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")
    
    save_dir_id = results[0]['id']
    shortcut_metadata = {
        'name': shortcut_name,
        'mimeType': 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {
            'targetId': share_id
        },
        'parents': [save_dir_id]
    }
    try:
        shortcut_file = drive_service.files().create(body=shortcut_metadata, fields='id').execute()
        shortcut_id = shortcut_file['id']
        print("Shortcut created successfully!")
    
    except HttpError as error:
        print("An error occurred: %s" % error)
        shortcut_id = None
    
    return shortcut_id

# /models/Stable-diffusion/*
def add_shortcut_files_to_drive (share_id, save_dir_name, save_dir):

    query = f"name = '{save_dir_name}' and mimeType = 'application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")
    
    save_dir_id = results[0]['id']
    shortcut_id = None

    query = f"'{share_id}' in parents and mimeType != 'application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    file_ids = [result['id'] for result in results]

    for file_id in file_ids:
        file = drive_service.files().get(fileId=file_id).execute()
        file_name = file['name']
        shortcut_metadata = {
            'name': file_name,
            'mimeType': 'application/vnd.google-apps.shortcut',
            'shortcutDetails': {
               'targetId': file_id
            },
            "parents": [save_dir_id]
        }
        shortcut_file = drive_service.files().create(body=shortcut_metadata).execute()
        shortcut_id = shortcut_file['id']
        print(f"Shortcut created for {file_name} with ID: {shortcut_id}")

# delete_shortcut /models/Stable-diffusion/*
def delete_shortcut_files(save_dir):

    folder_id = get_save_dir_id(save_dir)
    
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.shortcut'"
    results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
    shortcuts = results.get('files', [])
 
    if shortcuts:
        print(f"Found {len(shortcuts)} shortcuts in folder {folder_id}:")
        for shortcut in shortcuts:
            print(f"{shortcut['name']} ({shortcut['id']})")
        for shortcut in shortcuts:
            drive_service.files().delete(fileId=shortcut['id']).execute()
        print(f"Deleted {len(shortcuts)} shortcuts")
    else:
        print(f"No shortcuts found in folder {folder_id}")
#@markdown 
#@markdown 使用云端主模型 （By B站 邀月浮白）
shortcut_to_models = True #@param {type:"boolean"}

if shortcut_to_models and (not os.path.exists(gwebui_dir+"/models/Stable-diffusion"+ "/sdmodels")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("sdmodels", "16-eFzZcduNlu1c-ZHu9FE9FD4Jwi-6Er", "Stable-diffusion", save_dir)
  add_shortcut_directory_to_drive ("fbmodels", "1FuoMVN3GxaMP0J1mjA73SgmKWy_ELmXi", "Stable-diffusion", save_dir)

#@markdown 使用云端Lora模型 （By B站 邀月浮白）
shortcut_to_lora = True #@param {type:"boolean"}
if shortcut_to_lora and (not os.path.exists(gwebui_dir+"/models/Lora"+ "/fb_lora")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("fb_lora", "1ZFNyj3LERv8W10TxR1zOGQVt0vTdl6x9", "Lora", save_dir)

#@markdown 使用云端VAE （By B站 邀月浮白）
shortcut_to_VAE = True #@param {type:"boolean"}
if shortcut_to_VAE and (not os.path.exists(gwebui_dir+"/models/VAE"+ "/fb_vae")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_files_to_drive ("1mpByHLWajydSg5iE223iVWRYfAhndAKA", "VAE", save_dir)

#@markdown 使用云端模ControlNet （By B站 邀月浮白）
shortcut_to_cn = False #@param {type:"boolean"}
if shortcut_to_cn:
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(gcn_dir)
  delete_shortcut_files(save_dir)
  add_shortcut_files_to_drive ("1_sVbogR2El4FM-KvUf4YRNHCB1CpXvgO", "models", save_dir)

In [ ]:
#@title 👇 卸载SDwebui
#从Google Drive卸载SDwebui
#清空 gwebui_dir ，保留配置文件、图片输出文件和模型文件。
import os
import sys

import binascii
sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
sd_temp = '/content/drive/MyDrive/sd_temp'

!mkdir -p {sd_temp}
if os.path.exists(os.path.join(gwebui_dir, 'config.json')):
    !mv -f {gwebui_dir}/config.json {sd_temp}

if os.path.exists(os.path.join(gwebui_dir, 'ui-config.json')):
    !mv -f {gwebui_dir}/ui-config.json {sd_temp}

if os.path.exists(os.path.join(gwebui_dir, 'outputs')):
    !mv -f {gwebui_dir}/outputs {sd_temp}

if os.path.exists(os.path.join(gwebui_dir, 'scripts')):
    !mv -f {gwebui_dir}/scripts {sd_temp}

if os.path.exists(os.path.join(gwebui_dir, 'embeddings')):
    !mv -f {gwebui_dir}/embeddings {sd_temp}

if os.path.exists(os.path.join(gwebui_dir, 'models')):
    !mv -f {gwebui_dir}/models {sd_temp}
"""
if os.path.exists(os.path.join(gwebui_dir, 'extensions/{w}-additional-networks/models/lora')):
    !mv -f {gwebui_dir}/extensions/{w}-additional-networks/models/lora {sd_temp}
"""
if os.path.exists(gwebui_dir):
    !rm -rf {gwebui_dir}
    !mkdir -p {gwebui_dir} 

!mv -f {sd_temp}/* {gwebui_dir}

print("SDwebui uninstalled successfully.")